In [13]:
%reload_ext autoreload
%autoreload 2
# BASE
import os
import json
# COOL
from pyeed.core import ProteinRecord

In [14]:
# read in the protein records from the json
current_path = os.path.dirname(os.getcwd())
read_blast_folder = os.path.join(current_path, "TEM-lactamase", "data", "data_blast_search_big_list002_all_10")
# /home/niklas/Desktop/Job_Niklas/TEM-lactamase/data/data_blast_search_big_list001test_all_10
# list all the files in the directory
# read in the saved fetched results
protein_list = []
for file in os.listdir(read_blast_folder):
    with open(os.path.join(read_blast_folder, file), "r") as f:
        protein_list.append(ProteinRecord.from_json(f))

print('The fetched list of results is: ' +  str(len(protein_list)))

# now we check for double entries, we check the ids
count = 0
ids = []
filtered_protein_list = []

for protein in protein_list:
    if protein.id not in ids:
        ids.append(protein.id)
        filtered_protein_list.append(protein)
    else:
        count += 1

print('The number of double entries is: ' + str(count))
print('The number of filtered entries is: ' + str(len(filtered_protein_list)))

The fetched list of results is: 621
The number of double entries is: 544
The number of filtered entries is: 77


In [21]:
# read in the file from json
tem_dic = {}

with open(os.path.join(current_path, "TEM-lactamase", "TEM_Ids.json"), "r") as f:
    tem_dic = json.load(f)

# now read in the ids with the tem names and rewrite the list in a dic
filtered_protein_dic = {}

for protein in filtered_protein_list:
    # check if one of the regions has a region with a start name "blastquery:"
    found_in_region = False
    for region in protein.regions:
        if region.name == "blastquery":
            found_in_region = True
            filtered_protein_dic[tem_dic[region.name.split(":")[1]]] = protein
    if not found_in_region:
        filtered_protein_dic['-'+protein.id] = protein
            

print('The number of proteins in the filtered protein dic is: ' + str(len(filtered_protein_dic)))
print(filtered_protein_dic)


The number of proteins in the filtered protein dic is: 77
{'-P80545': ProteinRecord(id='P80545', name='Beta-lactamase', organism=Organism(id='b9efd5aa-2609-4117-bbdd-0726835de138', taxonomy_id=47917, name='Serratia fonticola', domain='Bacteria', kingdom=None, phylum='Pseudomonadota', tax_class='Gammaproteobacteria', order='Enterobacterales', family='Yersiniaceae', genus='Serratia', species=None, json_ld_type=['Organism'], json_ld_context={'Organism': 'https://github.com/PyEED/pyeed/Organism', 'taxonomy_id': 'http://edamontology.org/data_1179', 'name': 'http://edamontology.org/data_2909', 'kingdom': 'http://edamontology.org/data_1044', 'family': 'http://edamontology.org/data_2732', 'genus': 'http://edamontology.org/data_1870', 'species': 'http://edamontology.org/data_1045'}), sequence='QPANAKANIQQQLSELEKNSGGRLGVALIDTADNSQILYRGDERFPMCSTSKVMAVSALLKQSETDKNLLAKRMEIKQSDLVNYNPIAEKHLDTGMTLAEFSAATIQYSDNTAMNKILEHLGGPAKVTEFARTIGDKTFRLDRTEPTLNTAIPGDKRDTTSPQAMAISLQNLTLGKALAEPQRAQLVEWMKGNTTGGASIRAGL

In [22]:
# now we run clustal omega on the proteins
from pyeed.tools.clustalo import ClustalOmega

sequences = [protein.sequence for protein in filtered_protein_dic.values()]

# fix the format for clustal omega
# sequences = [
#     ">seq1\nMTHKLLLTLLFTLLFSSAYSRG",
#     ">seq2\nMTHKILLLTLLFTLLFSSAYSRG",
#     ">seq3\nMTHKILLLTLLFTLLFSSCYSRG",
# ]
sequences_filtered = []

for i, seq in enumerate(sequences):
    sequences_filtered.append(">seq" + str(i) + "\n" + seq)

print(sequences_filtered)

clustalo = ClustalOmega()
alignment = clustalo.align(sequences_filtered)


['>seq0\nQPANAKANIQQQLSELEKNSGGRLGVALIDTADNSQILYRGDERFPMCSTSKVMAVSALLKQSETDKNLLAKRMEIKQSDLVNYNPIAEKHLDTGMTLAEFSAATIQYSDNTAMNKILEHLGGPAKVTEFARTIGDKTFRLDRTEPTLNTAIPGDKRDTTSPQAMAISLQNLTLGKALAEPQRAQLVEWMKGNTTGGASIRAGLPTTWVVGDKTGSGDYGTTNDIAVIWPANHAPLVLVTYFTQPQQNAEARKDVLAAAAKIVTAGL', '>seq1\nMKHSSLRRSLLLAGITLPLVSFALPAWANALPASVDKQLAELERNANGRLGVAMINTGNGTKILYRAAQRFPFCSTFKFMLAAAVLDQSQSQPNLLNKHINYHESDLLSYAPITRKNLAHGMTVSELCAATIQYSDNTAANLLIKELGGLAAVNQFARSIGDQMFRLDRWEPDLNTARPNDPRDTTTPAAMAASMNKLVLGDALRPAQRSQLAVWLKGNTTGDATIRAGAPTDWIVGDKTGSGDYGTTNDIAVLWPTKGAPIVLVVYFTQREKDAKPRRDVLASVTKIILSQIS', '>seq2\nMIKSSWRKIAMLAAAVPLLLASGALWASTDAIHQKLTDLEKRSGGRLGVALINTADNSQILYRGDERFAMCSTSKVMAAAAVLKQSESNKEVVNKRLEINAADLVVWSPITEKHLQSGMTLAELSAATLQYSDNTAMNLIIGYLGGPEKVTAFARSIGDATFRLDRTEPTLNTAIPGDERDTSTPLAMAESLRKLTLGDALGEQQRAQLVTWLKGNTTGGQSIRAGLPESWVVGDKTGAGDYGTTNDIAVIWPEDHAPLVLVTYFTQPQQDAKNRKEVLAAAAKIVTEGL', '>seq3\nMRYIRLCIISLLATLPLAVHASPQPLEQIKLSESQLSGRVGMIEMDLASGRTLTAWRADERFPMMSTFKVVLCGAVLARVDAGDEQLERKIHYRQQDLVDYSPVSEK

KeyboardInterrupt: 